### Import libraries

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
from pulp import *
import json
import aux_functions_ml as f

In [6]:
df_inventory = pd.read_csv('../Datasets/clean_inventory.csv', index_col=0)
df_order_detail = pd.read_csv('../Datasets/clean_order_details.csv', index_col = 0)
df_products = pd.read_csv('../Datasets/clean_products.csv', index_col=0)
df_inventory

,TransactionID,ProductID,PurchaseOrderID,MissingID,TransactionDate,UnitPurchasePrice,QuantityOrdered,QuantityReceived,QuantityMissing
1148,2119,1213,25,0.0,2003-10-15,5.8,2.0,2.0,0.0
1151,2370,330,27,0.0,2003-09-16,6.3,80.0,80.0,0.0
1152,2371,117,27,0.0,2003-09-16,6.7,80.0,80.0,0.0
1153,2372,769,27,0.0,2003-09-16,7.3,60.0,60.0,0.0
1154,2373,894,27,0.0,2003-09-16,7.9,80.0,80.0,0.0
...,...,...,...,...,...,...,...,...,...
20946,41219,4419,315,0.0,2006-04-05,1.3,10.0,10.0,0.0
20947,41220,4420,315,0.0,2006-04-05,1.2,10.0,10.0,0.0
20948,41221,4421,315,0.0,2006-04-05,1.4,10.0,10.0,0.0
20949,41222,4261,315,0.0,2006-04-05,6.0,30.0,30.0,0.0


In [7]:
df_order_detail

,OrderDetailID,OrderID,ProductID,QuantitySold,UnitSalesPrice,total_price
0,2,2,955,5,7.5,37.5
1,3,2,958,5,8.5,42.5
2,5,2,959,5,8.5,42.5
3,6,4,72,50,3.0,150.0
4,7,4,9,50,3.2,160.0
...,...,...,...,...,...,...
105752,114665,2470,4342,20,1.7,34.0
105753,114666,2470,4419,10,1.9,19.0
105754,114667,2470,4343,20,1.9,38.0
105755,114668,2470,4421,10,2.1,21.0


In [8]:
stock_actual = df_inventory.groupby('ProductID')['QuantityReceived'].sum().reset_index()
stock_actual.columns = ['ProductID', 'StockActual']
stock_actual

,ProductID,StockActual
0,1,120.0
1,2,85.0
2,3,212.0
3,4,487.0
4,5,555.0
...,...,...
3992,4418,10.0
3993,4419,10.0
3994,4420,10.0
3995,4421,10.0


In [9]:
ventas_agrupadas = df_order_detail.groupby('ProductID')['QuantitySold'].sum().reset_index()
ventas_agrupadas

,ProductID,QuantitySold
0,1,134
1,2,134
2,3,241
3,4,512
4,5,624
...,...,...
4037,4419,10
4038,4420,10
4039,4421,10
4040,4422,5


In [10]:
stock_actual = stock_actual.merge(ventas_agrupadas, on="ProductID", how="left").fillna(0)
stock_actual

,ProductID,StockActual,QuantitySold
0,1,120.0,134.0
1,2,85.0,134.0
2,3,212.0,241.0
3,4,487.0,512.0
4,5,555.0,624.0
...,...,...,...
3992,4418,10.0,57.0
3993,4419,10.0,10.0
3994,4420,10.0,10.0
3995,4421,10.0,10.0


In [11]:
stock_actual['Stock'] = np.where(stock_actual['StockActual'] > stock_actual['QuantitySold'], 
                                 stock_actual['StockActual'] - stock_actual['QuantitySold'], 
                                 stock_actual['StockActual'])
stock_actual

,ProductID,StockActual,QuantitySold,Stock
0,1,120.0,134.0,120.0
1,2,85.0,134.0,85.0
2,3,212.0,241.0,212.0
3,4,487.0,512.0,487.0
4,5,555.0,624.0,555.0
...,...,...,...,...
3992,4418,10.0,57.0,10.0
3993,4419,10.0,10.0,10.0
3994,4420,10.0,10.0,10.0
3995,4421,10.0,10.0,10.0


In [12]:
stock_actual.drop(columns=['StockActual', 'QuantitySold'], inplace=True)
stock_actual

,ProductID,Stock
0,1,120.0
1,2,85.0
2,3,212.0
3,4,487.0
4,5,555.0
...,...,...
3992,4418,10.0
3993,4419,10.0
3994,4420,10.0
3995,4421,10.0


### Load map warehouse

In [13]:
with open("product_slot.json", "r") as file:
    product_slots = json.load(file)
    
product_slots

{'6': [0, 0],
 '7': [0, 1],
 '1034': [0, 2],
 '3614': [0, 3],
 '3615': [0, 4],
 '3616': [0, 5],
 '3617': [0, 6],
 '3618': [0, 7],
 '4180': [0, 8],
 '4140': [0, 9],
 '4141': [0, 10],
 '4142': [0, 11],
 '4143': [0, 12],
 '4144': [0, 13],
 '4145': [0, 14],
 '4146': [0, 15],
 '4147': [0, 16],
 '4148': [0, 17],
 '4149': [0, 18],
 '1078': [0, 19],
 '1079': [0, 20],
 '1080': [0, 21],
 '4150': [0, 22],
 '4151': [0, 23],
 '4152': [0, 24],
 '4153': [0, 25],
 '1597': [0, 26],
 '4154': [0, 27],
 '4155': [0, 28],
 '4156': [0, 29],
 '4157': [0, 30],
 '2114': [0, 31],
 '4158': [0, 32],
 '68': [0, 33],
 '69': [0, 34],
 '4159': [0, 35],
 '4160': [0, 36],
 '4161': [0, 37],
 '4162': [0, 38],
 '4163': [0, 39],
 '4164': [0, 40],
 '4165': [0, 41],
 '4166': [0, 42],
 '4167': [0, 43],
 '4187': [0, 44],
 '4188': [0, 45],
 '4178': [0, 46],
 '1108': [0, 47],
 '1109': [0, 48],
 '1110': [0, 49],
 '1111': [0, 50],
 '1112': [0, 51],
 '1113': [0, 52],
 '1114': [0, 53],
 '1115': [0, 54],
 '1116': [0, 55],
 '1117': [0,

In [15]:
df_slots = pd.DataFrame.from_dict(product_slots, orient='index', columns=['X', 'Y']).reset_index()
df_slots.rename(columns={'index': 'ProductID'}, inplace=True)
df_slots

,ProductID,X,Y
0,6,0,0
1,7,0,1
2,1034,0,2
3,3614,0,3
4,3615,0,4
...,...,...,...
4163,3555,45,41
4164,3572,55,131
4165,3064,55,132
4166,4091,113,108


In [16]:
stock_actual

,ProductID,Stock
0,1,120.0
1,2,85.0
2,3,212.0
3,4,487.0
4,5,555.0
...,...,...
3992,4418,10.0
3993,4419,10.0
3994,4420,10.0
3995,4421,10.0


In [17]:
df_slots["ProductID"] = df_slots["ProductID"].astype(str) 
stock_actual["ProductID"] = stock_actual["ProductID"].astype(str) 
df_products['ProductID'] = df_products['ProductID'].astype(str)
df_products = df_products[['ProductID', 'ProductName', 'Category', 'Gender', 'ProductLine', 'Weight', 'Size', 'PackSize', 'ProductType']]

In [18]:
df_slots = df_slots.merge(stock_actual, on="ProductID", how="left")  
df_slots = df_slots.merge(df_products, on = 'ProductID')
df_slots

,ProductID,X,Y,Stock,ProductName,Category,Gender,ProductLine,Weight,Size,PackSize,ProductType
0,6,0,0,296.0,3-BX34,Boxers,Boys' Briefs,Underwear,55,M,Single Unit,BX34
1,7,0,1,2.0,3-BX64,Boxers,Boys' Briefs,Underwear,55,M,Single Unit,BX64
2,1034,0,2,313.0,L-BX46A,Boxers,Men's Briefs,Underwear,80,L,Single Unit,BX46A
3,3614,0,3,20.0,2-BX46SL,Boxers,Boys' Briefs,Underwear,55,S,Single Unit,BX46SL
4,3615,0,4,8.0,3-BX46SL,Boxers,Boys' Briefs,Underwear,65,M,Single Unit,BX46SL
...,...,...,...,...,...,...,...,...,...,...,...,...
4163,3555,45,41,30.0,S-TB4602SY,Thongs,Women's Panties,Underwear,175,S,Dozen,TB4602SY
4164,3572,55,131,26.0,4046-S6D503,Regular-Over the Calf,Men's Socks,Socks,900,40-46,Dozen,S6D503
4165,3064,55,132,30.0,S-IS4619,Briefs,Women's Panties,Underwear,190,S,Dozen,IS4619
4166,4091,113,108,10.0,M-TB46PB,Thongs,Women's Panties,Underwear,170,M,Dozen,TB46PB


In [19]:
df_slots.to_csv("../Datasets/final_distribution_warehouse.csv")

### minimize distance

In [84]:
df_slots.tail(50)

,ProductID,X,Y,Stock,ProductName,Category,Gender,ProductLine,Weight,Size,PackSize,ProductType
4118,798,31,188,2.0,XL-BX46CB,Boxers,Men's Briefs,Underwear,76,XL,Single Unit,BX46CB
4119,4381,31,189,400.0,XL-S4628S,Briefs,Men's Briefs,Underwear,400,XL,Dozen,S4628S
4120,4382,31,190,0.0,XXL-S4628S,Briefs,Women's Panties,Underwear,495,XXL,Dozen,S4628S
4121,801,31,191,NaN,XL-BX64P,Boxers,Men's Briefs,Underwear,78,XL,Single Unit,BX64P
4122,4393,31,192,NaN,XXL-207,Undershirts,Women's Undershirts,Underwear,1150,XXL,Dozen,207
4123,4394,31,193,NaN,M-199,Undershirts,Women's Undershirts,Underwear,1020,M,Dozen,199
4124,1838,31,194,NaN,L-K551,Briefs,Women's Panties,Underwear,450,L,Dozen,K551
4125,1327,31,195,1.0,3XL-PCL24S,Drawers,Women's Panties,Underwear,1217,3XL,Dozen,PCL24S
4126,1839,31,196,NaN,XL-K551,Briefs,Women's Panties,Underwear,516,XL,Dozen,K551
4127,2354,31,197,0.0,9-S1D204,Regular-Over the Calf,Children's Socks,Socks,525,9,Dozen,S1D204


In [20]:
productos_a_recoger = {'L-BX46A': 5, 
                       'L-BX34D': 10,
                       'XXL-S4628S': 35, 
                       '9-S1D204': 23}

In [21]:
productos_a_recoger.keys()

dict_keys(['L-BX46A', 'L-BX34D', 'XXL-S4628S', '9-S1D204'])

In [23]:
df_slots = f.pickup_products(productos_a_recoger, df_slots)

     ProductID   X    Y  Stock ProductName               Category  \
2         1034   0    2  313.0     L-BX46A                 Boxers   
49        1110   0   49    2.0     L-BX34D                 Boxers   
4120      4382  31  190  156.0  XXL-S4628S                 Briefs   
4127      2354  31  197   20.0    9-S1D204  Regular-Over the Calf   

                Gender ProductLine  Weight Size     PackSize ProductType  
2         Men's Briefs   Underwear      80    L  Single Unit       BX46A  
49        Men's Briefs   Underwear      68    L  Single Unit       BX34D  
4120   Women's Panties   Underwear     495  XXL        Dozen      S4628S  
4127  Children's Socks       Socks     525    9        Dozen      S1D204  

Productos que se van a recoger (ajustados):
     ProductName  CantidadRecoger  Stock
2        L-BX46A              5.0  313.0
49       L-BX34D              2.0    2.0
4120  XXL-S4628S             35.0  156.0
4127    9-S1D204             20.0   20.0
Stock Actualizado 

Productos

c:\Users\Carlota\OneDrive\Documentos\Bootcamp Data Analysis\final-proyect\ML\aux_functions_ml.py:479: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  productos_disponibles['CantidadRecoger'] = productos_disponibles['ProductName'].apply(
